<a href="https://colab.research.google.com/github/shreyanshp/AA_Colab_Notebook_repo/blob/master/Keras_Transfer_Learning_TensorflowJS_MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


In [2]:
HEIGHT = 224
WIDTH = 224

base_model = MobileNetV2(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))

Instructions for updating:
Colocations handled automatically by placer.
9412608/9406464 [==============================] - 0s 0us/step


In [5]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'food_dataset/'

Mounted at /content/gdrive


In [6]:
TRAIN_DIR = base_dir
BATCH_SIZE = 8

train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True
    )

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE)

Found 6 images belonging to 3 classes.


In [7]:
def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

class_list = ["pizza", "burger", "taco"]
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=len(class_list))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
NUM_EPOCHS = 1
BATCH_SIZE = 8
num_train_images = 10000

adam = Adam(lr=0.00001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])


history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=8, 
                                       steps_per_epoch=num_train_images // BATCH_SIZE, 
                                       shuffle=True)

!rm -rf ./mymobilenet* ./model.h5
finetune_model.save('./model.h5')
    

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
1250/1250 [==============================] - 151s 121ms/step - loss: 0.0559 - acc: 0.9793


In [9]:
!pip install tensorflowjs

    100% |████████████████████████████████| 13.9MB 2.5MB/s 
    100% |████████████████████████████████| 81kB 26.2MB/s 
    100% |████████████████████████████████| 96.1MB 378kB/s 
    100% |████████████████████████████████| 358kB 19.8MB/s 
    100% |████████████████████████████████| 3.0MB 9.5MB/s 
    100% |████████████████████████████████| 61kB 24.6MB/s 
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6
  Found existing installation: tensorflow-hub 0.4.0
    Uninstalling tensorflow-hub-0.4.0:
      Successfully uninstalled tensorflow-hub-0.4.0


In [10]:
from google.colab import files
!tensorflowjs_converter --input_format=keras ./model.h5 ./mymobilenet
!zip -r ./mymobilenet.zip ./mymobilenet 
files.download( "./mymobilenet.zip" )

  adding: mymobilenet/ (stored 0%)
  adding: mymobilenet/group1-shard61of65.bin (deflated 8%)
  adding: mymobilenet/group1-shard27of65.bin (deflated 8%)
  adding: mymobilenet/group1-shard13of65.bin (deflated 8%)
  adding: mymobilenet/group1-shard20of65.bin (deflated 8%)
  adding: mymobilenet/group1-shard35of65.bin (deflated 8%)
  adding: mymobilenet/group1-shard51of65.bin (deflated 8%)
  adding: mymobilenet/group1-shard11of65.bin (deflated 8%)
  adding: mymobilenet/group1-shard18of65.bin (deflated 8%)
  adding: mymobilenet/group1-shard23of65.bin (deflated 8%)
  adding: mymobilenet/group1-shard31of65.bin (deflated 8%)
  adding: mymobilenet/model.json (deflated 95%)
  adding: mymobilenet/group1-shard38of65.bin (deflated 8%)
  adding: mymobilenet/group1-shard34of65.bin (deflated 8%)
  adding: mymobilenet/group1-shard2of65.bin (deflated 7%)
  adding: mymobilenet/group1-shard62of65.bin (deflated 8%)
  adding: mymobilenet/group1-shard47of65.bin (deflated 8%)
  adding: mymobilenet/group1-shar

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 48110, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil